In [16]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [17]:
weather = pd.read_csv('weather.csv')
print(weather.shape)

(96453, 10)


In [18]:
weather.head()

,Formatted Date,Precip Type,Summary,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Pressure (millibars)
0,2006-04-01 00:00:00.000 +0200,rain,Partly Cloudy,9.472222,7.388889,0.89,14.1197,251,15.8263,1015.13
1,2006-04-01 01:00:00.000 +0200,rain,Partly Cloudy,9.355556,7.227778,0.86,14.2646,259,15.8263,1015.63
2,2006-04-01 02:00:00.000 +0200,rain,Mostly Cloudy,9.377778,9.377778,0.89,3.9284,204,14.9569,1015.94
3,2006-04-01 03:00:00.000 +0200,rain,Partly Cloudy,8.288889,5.944444,0.83,14.1036,269,15.8263,1016.41
4,2006-04-01 04:00:00.000 +0200,rain,Mostly Cloudy,8.755556,6.977778,0.83,11.0446,259,15.8263,1016.51


In [19]:
weather = weather.dropna()
outcome=weather.pop("Precip Type")

In [20]:
final = weather.drop(['Formatted Date'], axis='columns')
final

,Summary,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Pressure (millibars)
0,Partly Cloudy,9.472222,7.388889,0.89,14.1197,251,15.8263,1015.13
1,Partly Cloudy,9.355556,7.227778,0.86,14.2646,259,15.8263,1015.63
2,Mostly Cloudy,9.377778,9.377778,0.89,3.9284,204,14.9569,1015.94
3,Partly Cloudy,8.288889,5.944444,0.83,14.1036,269,15.8263,1016.41
4,Mostly Cloudy,8.755556,6.977778,0.83,11.0446,259,15.8263,1016.51
...,...,...,...,...,...,...,...,...
96448,Partly Cloudy,26.016667,26.016667,0.43,10.9963,31,16.1000,1014.36
96449,Partly Cloudy,24.583333,24.583333,0.48,10.0947,20,15.5526,1015.16
96450,Partly Cloudy,22.038889,22.038889,0.56,8.9838,30,16.1000,1015.66
96451,Partly Cloudy,21.522222,21.522222,0.60,10.5294,20,16.1000,1015.95


In [21]:
from sklearn.preprocessing import LabelEncoder
daily_summary = LabelEncoder()
final['Summary_n'] = daily_summary.fit_transform(final['Summary'])

daily_outcome = LabelEncoder()
outcome['outcome_n'] = daily_outcome.fit_transform(outcome)

In [22]:
final.isna().any()

Summary                     False
Temperature (C)             False
Apparent Temperature (C)    False
Humidity                    False
Wind Speed (km/h)           False
Wind Bearing (degrees)      False
Visibility (km)             False
Pressure (millibars)        False
Summary_n                   False
dtype: bool

In [23]:
NAs = pd.concat([weather.isnull().sum()],axis=1, keys=["weather"])
NAs[NAs.sum(axis=1)>0]

,weather


In [24]:
#filling temp values with mean
weather["Temperature (C)"] = weather["Temperature (C)"].fillna(weather["Temperature (C)"].mean())

#filling Apparent Temperature values with mean
weather["Apparent Temperature (C)"] = weather["Apparent Temperature (C)"].fillna(weather["Apparent Temperature (C)"].mean())

#filling Humidity values with mean
weather["Humidity"] = weather["Humidity"].fillna(weather["Humidity"].mean())

#filling Wind Speed values with mean
weather["Wind Speed (km/h)"] = weather["Wind Speed (km/h)"].fillna(weather["Wind Speed (km/h)"].mean())

#filling Wind Bearing values with mean
weather["Wind Bearing (degrees)"] = weather["Wind Bearing (degrees)"].fillna(weather["Wind Bearing (degrees)"].mean())

#filling Visibility values with mean
weather["Visibility (km)"] = weather["Visibility (km)"].fillna(weather["Visibility (km)"].mean())

#filling Pressure values with mean
weather["Pressure (millibars)"] = weather["Pressure (millibars)"].fillna(weather["Pressure (millibars)"].mean())


In [25]:
data = final.drop(['Summary'], axis='columns')


In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, outcome['outcome_n'], test_size=0.25)

In [27]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
print(y_test[0:40])

[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 1]


In [28]:
!pip install scikit-learn

In [29]:
import scikitplot as skplt
skplt.estimators.plot_learning_curve(rf,x_train, y_train)
plt.show()

ModuleNotFoundError: ignored

In [ ]:
col=['Temperature (C)', 'Apparent Temperature (C)', 'Humidity', 'Wind Speed (km/h)', 'Wind Bearing (degrees)', 'Visibility (km)', 'Pressure (millibars)',"Summary_n"]
col

In [ ]:
feature_imp = pd.Series(rf.feature_importances_,index=data.columns).sort_values(ascending=False)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Creating a bar plot
sns.barplot(x=feature_imp, y=feature_imp.index)
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()

In [ ]:
# import os
# from sklearn.tree import export_graphviz
# import six
# import pydot
# from sklearn import tree
# dotfile = six.StringIO()
# i_tree = 0
# for tree_in_forest in rf.estimators_:
#     export_graphviz(tree_in_forest,out_file='tree.dot',
#     feature_names=data.columns,
#     filled=True,
#     rounded=True)
#     (graph,) = pydot.graph_from_dot_file('tree.dot')
#     name = 'tree' + str(i_tree)
#     graph.write_png(name+  '.png')
#     os.system('dot -Tpng tree.dot -o tree.png')
#     i_tree +=1

In [ ]:
# from sklearn.tree import export_graphviz
# from subprocess import call
# from IPython.display import Image

# tree_in_forest = rf.estimators_[5]

# # Export as dot file
# export_graphviz(tree_in_forest, out_file='tree.dot', 
# feature_names = col,
# class_names =["rain","snow"],
# rounded = True, proportion = False, 
# precision = 2, filled = True)
# # Convert to png using system command (requires Graphviz)
# call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])
# # Display in jupyter notebook
# Image(filename = 'tree.png')



In [ ]:
y_pred = rf.predict(x_test)
print(y_pred[0:40])

In [ ]:
rf.score(x_test, y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
matrix = confusion_matrix(y_test, y_pred)
print('Confusion matrix : \n',matrix)

In [ ]:
tp, fn, fp, tn = confusion_matrix(y_test, y_pred).reshape(-1)

print('Outcome values : \n', tp, fn, fp, tn)
 	 
# classification report for precision, recall f1-score and accuracy
matrix = classification_report(y_test, y_pred)
print('Classification report : \n',matrix)

In [ ]:
new=pd.DataFrame([[0.00066, 2.1111, 0.6, 98.73, 1, 0.62634, 1021.26, 13]],columns=col)
new

In [ ]:
rf.predict(new)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
plt.figure(figsize=(10,7))
sn.heatmap(matrix, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')